In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import glob
import cv2
import numpy as np

from tqdm import tqdm
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/siddharth/anaconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/siddharth/anaconda3/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### Class to load the images

In [2]:
class ImageProvider():
    def __init__(self, image_paths):
        
        self.images = []
        for path in image_paths:
            
            img = cv2.imread(path)
            resized_img = cv2.resize(img, (224,224), interpolation=cv2.INTER_AREA)
            self.images.append(resized_img)
    
    def getImages(self):

        return self.images

### Class to convert the images to correct size 

In [3]:
class CustomDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]

        if self.transform:
            image = self.transform(image)

        return image

### Data class for actual training and testing

In [4]:
class Data(Dataset):
    def __init__(self,data):

        n = data.shape[1]
        self.features = torch.tensor(data.iloc[:, 0:n-1].values.astype(np.int64), dtype=torch.float32)
        self.labels = torch.tensor(data.iloc[:, -1].values.astype(np.int64), dtype=torch.int64)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

    def __len__(self):
        return len(self.features)

### Classifier model class

In [5]:
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout:float = 0.3):
        super().__init__()

        self.layer1 = self.layer1 = nn.Sequential(
            nn.Linear(in_features=input_size, out_features=hidden_size),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout),
        )
        
        self.output_layer = nn.Linear(in_features=hidden_size, out_features=output_size)

    def forward(self, x):
        
        x = self.layer1(x)
        x = self.output_layer(x)

        return x

### Alexnet model parameters

In [6]:
# Load pre-trained AlexNet model
alexnet_model = models.alexnet(pretrained=True)
alexnet_model.eval()  # Set the model to evaluation mode

# Remove the classification layer of AlexNet
alexnet_model = nn.Sequential(*list(alexnet_model.children())[:-1])

# Freeze the parameters of the feature extractor
for param in alexnet_model.parameters():
    param.requires_grad = False

/home/siddharth/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/siddharth/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Getting the images and labels

In [7]:
objects = ["Bikes", "Horses"]

image_paths = []  
labels = []
label = 0

for object in objects:
    string = f"./BikeHorses/{object}/*.jpg"

    files = glob.glob(string, recursive = True)
    image_paths.extend(files)

    labels_temp = [label]*len(files)
    labels.extend(labels_temp)

    label+=1

print(image_paths)
print(labels)

['./BikeHorses/Bikes/0066.jpg', './BikeHorses/Bikes/0052.jpg', './BikeHorses/Bikes/0031.jpg', './BikeHorses/Bikes/0046.jpg', './BikeHorses/Bikes/0018.jpg', './BikeHorses/Bikes/0045.jpg', './BikeHorses/Bikes/0064.jpg', './BikeHorses/Bikes/0049.jpg', './BikeHorses/Bikes/0070.jpg', './BikeHorses/Bikes/0061.jpg', './BikeHorses/Bikes/0032.jpg', './BikeHorses/Bikes/0050.jpg', './BikeHorses/Bikes/0010.jpg', './BikeHorses/Bikes/0047.jpg', './BikeHorses/Bikes/0044.jpg', './BikeHorses/Bikes/0053.jpg', './BikeHorses/Bikes/0062.jpg', './BikeHorses/Bikes/0038.jpg', './BikeHorses/Bikes/0072.jpg', './BikeHorses/Bikes/0063.jpg', './BikeHorses/Bikes/0043.jpg', './BikeHorses/Bikes/0019.jpg', './BikeHorses/Bikes/0012.jpg', './BikeHorses/Bikes/0079.jpg', './BikeHorses/Bikes/0054.jpg', './BikeHorses/Bikes/0020.jpg', './BikeHorses/Bikes/0008.jpg', './BikeHorses/Bikes/0056.jpg', './BikeHorses/Bikes/0035.jpg', './BikeHorses/Bikes/0057.jpg', './BikeHorses/Bikes/0080.jpg', './BikeHorses/Bikes/0075.jpg', './Bike

### Converting the images to features from AlexNet

In [8]:
provider = ImageProvider(image_paths)

reqd_images = provider.getImages()

# Define transformations for preprocessing images
transform = transforms.Compose([
    transforms.ToTensor()
])

full_dataset = CustomDataset(reqd_images, transform=transform)
full_loader = DataLoader(full_dataset, batch_size=1, shuffle=False)

In [9]:
# Extract features for all images
all_features = []

with torch.no_grad():
    for images in full_loader:
        features = alexnet_model(images)
        features = np.reshape(features.squeeze(), -1)
        all_features.append(features)

all_features = np.array(all_features)
print(all_features.shape)
labels = np.array(labels)
print(labels.shape)

(179, 9216)
(179,)


### Train Test Split to train model

In [10]:
X_train, X_test, y_train, y_test = train_test_split(all_features, labels, test_size=0.2, random_state=42)

In [11]:
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(y_train)
X_test = pd.DataFrame(X_test)
Y_test = pd.DataFrame(y_test)

X_train = pd.concat([X_train, Y_train], axis=1)
X_test = pd.concat([X_test, Y_test], axis=1)

In [12]:
train_dataset = Data(data=X_train)
test_dataset = Data(data=X_test)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=True)

### Parameters of the model

In [13]:
input_size = all_features.shape[1]
print(input_size)
hidden_size = 256  # Adjust this as needed
num_classes = 2

classifier = Classifier(input_size, hidden_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

sanity_check=False
n_epochs = 5

9216


### Helper functions for Training and Validation

In [14]:
def train_epoch(model, dataloader, optimiser, criterion):
    model.train()

    for batch in tqdm(dataloader):
        x, y = batch[0], batch[1]

        output = model(x)
        output = nn.Softmax(dim=-1)(output)
        loss = criterion(output, y)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        if sanity_check:
            break

In [15]:
def validate(
    model,
    dataloader,
    criterion
):
    model.eval()
    total_loss = 0
    predictions = []
    truths = []

    with torch.no_grad():
        for batch in tqdm(dataloader):
            x, y = batch[0], batch[1]

            output = model(x)
            output = nn.Softmax(dim=-1)(output)
            loss = criterion(output, y)
            total_loss += loss.detach().cpu().item()/len(dataloader)

            preds = torch.argmax(output, dim=-1)
            predictions.extend(preds.cpu())
            truths.extend(y.cpu())

            if sanity_check:
                break

    acc = accuracy_score(y_true=truths, y_pred=predictions)
    f1 = f1_score(y_true=truths, y_pred=predictions, average='macro')

    return total_loss, acc, f1

In [16]:
def train_model(
    model,
    train_dataloader,
    test_dataloader,
    optimiser,
    criterion
):
    for epoch in range(1, n_epochs+1):
        start_time = time.time()

        print(f"========= EPOCH {epoch} STARTED =========")
        train_epoch(model=model, dataloader=train_dataloader, optimiser=optimiser, criterion=criterion)

        print(f"========= TRAIN EVALUATION STARTED =========")
        train_val_op = validate(model=model, dataloader=train_dataloader, criterion=criterion)

        print(f"========= TEST EVALUATION STARTED =========")
        test_val_op = validate(model=model, dataloader=test_dataloader, criterion=criterion)

        print(f"END OF {epoch} EPOCH")
        print(f"| Time taken: {time.time() - start_time: 7.3f} |")
        print(f"| Train Loss: {train_val_op[0]: 7.3f} | Train acc: {train_val_op[1]: 1.5f} | Train f1: {train_val_op[2]: 1.5f} |")
        print(f"| Test Loss: {test_val_op[0]: 7.3f}  | Test acc: {test_val_op[1]: 1.5f}  | Test f1: {test_val_op[2]: 1.5f}  |")

        if sanity_check:
            break

### Train the model and report accuracies

In [18]:
train_model(
    model=classifier,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimiser=optimizer,
    criterion=criterion
)

========= EPOCH 1 STARTED =========


100%|██████████| 2/2 [00:00<00:00,  7.45it/s]


========= TRAIN EVALUATION STARTED =========


100%|██████████| 2/2 [00:00<00:00, 18.89it/s]


========= TEST EVALUATION STARTED =========


...

========= TRAIN EVALUATION STARTED =========


100%|██████████| 2/2 [00:00<00:00, 22.89it/s]


========= TEST EVALUATION STARTED =========


100%|██████████| 1/1 [00:00<00:00, 167.32it/s]

END OF 5 EPOCH
| Time taken:   0.616 |
| Train Loss:   0.314 | Train acc:  1.00000 | Train f1:  1.00000 |
| Test Loss:   0.317  | Test acc:  1.00000  | Test f1:  1.00000  |
